# PointNet Transfer Learning on subset of ModelNet10


The ModelNet10 dataset contains 4899 3D Meshes from 10 different classes.

The classes are:

* bed
* monitor
* desk
* chair
* dresser
* toilet
* sofa
* table
* night stand
* bathttub 

We provide you a pretrained version of PointNet. It was trained on 6 of the above 10 classes. Your task is to implement the PointNet Model and train it to classify the other 4 classes using transfer learning.

The classes to be learnt are:

* desk
* chair
* toilet 
* table



Tasks:
* implement the TODOs
* transfer learn the PointNet to achieve an Accuracy > 90%
Help:
* use the PointNet Paper [clickedy](http://stanford.edu/~rqi/pointnet/)
* use the Keras API Documentation [clickedy](https://www.keras.io/)

In [ ]:
!pip install open3d

In [ ]:
%load_ext tensorboard

In [ ]:
import open3d as o3d
import os

from tensorflow.keras.utils import Sequence, to_categorical

import numpy as np
import pandas as pd
import mlflow

%load_ext autoreload
%matplotlib inline
import numpy as np

In [ ]:
%%sh
# download ModelNet10
wget -q http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
# unpack
unzip -oq ModelNet10.zip
# remove Archive
rm ModelNet10.zip

In [ ]:
KEEP = ["chair", "desk", "toilet", "table"]
classes = [f for f in os.scandir("ModelNet10/") if os.path.isdir(f)]

import shutil

for c in classes:
    if c.name not in KEEP:
        shutil.rmtree(c.path, ignore_errors=False, onerror=None)

##  Part One: Data Provider

We use the Keras Sequence API to construct a data provider, which feeds our model during training and validation. Fill in all the ToDos to make it work.

In [ ]:
class ModelNetProvider(Sequence):
    """
    Lazily load point clouds and annotations from filesystem and prepare it for model training.
    """

    def __init__(self, dataset, batch_size, n_classes, sample_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.sample_size = sample_size

        # indices of samples used for shuffling
        self.indices = # ToDo

        self.on_epoch_end()

    def __len__(self):
        return  # ToDo: Define the length of the generator (Hint: Number of Steps in one Epoch)

    def __getitem__(self, index):
        """Generate one batch of data."""
        batch_indices = # ToDo: select the indices for the current batch, starting at `index`
        batch_samples = self.dataset.iloc[batch_indices]

        return self.__generate_data(batch_samples)

    def __generate_data(self, batch_samples):
        X = []
        y = []
        for i, row in batch_samples.iterrows():
            mesh = o3d.io.read_triangle_mesh((row["path"]))
            pcd = mesh.sample_points_uniformly(number_of_points=self.sample_size)
            points = np.asarray(pcd.points)
            centered_points = # ToDo center the points to origin
            normalized_points = # ToDo normalize the centered points
            X.append(normalized_points)
            y.append(row["class"])

        return self.rotate_point_clouds(np.array(X)), to_categorical(np.array(y), num_classes=self.n_classes)

    def on_epoch_end(self):
        """Shuffle training data, so batches are in different order"""
        np.random.shuffle(self.indices)
        
        
    def rotate_point_clouds(self, batch, rotation_angle_range=(-np.pi/8, np.pi/8)):
        """Rotate point cloud around y-axis (=up) by random angle"""
        for b, pc in enumerate(batch):
            phi = np.random.uniform(*rotation_angle_range)
            c, s = np.cos(phi), np.sin(phi)
            R = np.matrix([[c, 0, s],
                           [0, 1, 0],
                           [-s, 0, c]])
            batch[b, :, :3] = np.dot(pc[:, :3], R)
        return batch

In [ ]:
def initialize_dataset(data_directory, file_extension=".off"):
    """
    Loads an index to all files and structures them.
    :param data_directory: directory containing the data files
    :param file_extension: extension of the data files
    :return: pandas dataframe containing an index to all files and a label index, 
        mapping numerical label representations to label names.
    """
    files = [
        os.path.join(r, f)  
            for r, d, fs in os.walk(data_directory) 
            for f in fs if f.endswith(file_extension)
        ]
    
    dataframe = pd.DataFrame({
        "path": files,
        "class": pd.Categorical([f.rsplit("/", 3)[1] for f in files]),
        "is_train": ["train" in f for f in files]
    })
    
    factorization = dataframe["class"].factorize()
    dataframe["class"] = factorization[0]
    
    return dataframe, factorization[1]


## Part Two: PointNet Architecture

Implement all the missing code pieces to complete the whole architecture. Follow the description in the paper or use the image below. If your model definition is right, you should be able to load our pretrained weigths without an error. 

We provided the definition of the T-Net, so you don't need to implement it on your own. If you are interested in the implementation details, just have a look at the pointnet_utils.py file.

![PointNet Architecture](http://stanford.edu/~rqi/pointnet/images/pointnet.jpg)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Dot, Lambda, \
    Reshape, concatenate, GlobalMaxPooling1D, BatchNormalization, \
    Activation, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.regularizers import Regularizer

import numpy as np

### Utility functions for convolutions and densely connected layers with batch normalization


In [ ]:
def conv1d_bn(x, num_filters, kernel_size, padding='same', strides=1,
              use_bias=False, scope=None, activation='relu'):
    """
    Utility function to apply Convolution + Batch Normalization.
    """
    with K.name_scope(scope):
        input_shape = x.get_shape().as_list()[-2:]
        x = # ToDo define the Convolutional Part of this Layer
        x = # ToDo add BatchNormalization
        x = # ToDo add activation function
    return x


def dense_bn(x, units, use_bias=True, scope=None, activation=None):
    """
    Utility function to apply Dense + Batch Normalization.
    """
    with K.name_scope(scope):
        x = # ToDo Add Dense Part of this Layer
        x = # ToDo add BatchNormalization
        x = # ToDo add activation function
    return x

### Own Regularizer

In [ ]:
class OrthogonalRegularizer(Regularizer):
    """
    Considering that input is flattened square matrix X, regularizer tries to ensure that matrix X
    is orthogonal, i.e. ||X*X^T - I|| = 0. L1 and L2 penalties can be applied to it
    """    
    def __init__(self, l1=0.0, l2=0.0):
        self.l1 = K.cast_to_floatx(l1)
        self.l2 = K.cast_to_floatx(l2)

    def __call__(self, x):
        size = int(np.sqrt(x.shape[1].value))
        assert size * size == x.shape[1].value
        x = K.reshape(x, (-1, size, size))
        xxt = # ToDo compute dot product on the batches
        regularization = 0.0
        if self.l1:
            regularization += # ToDo: Implement L1 regularization
        if self.l2:
            regularization += # ToDo: Implement L2 regularization

        return regularization
    
    def get_config(self):
        return {'l1': float(self.l1), 'l2': float(self.l2)}
    

    

def orthogonal(l1=0.0, l2=0.0):
    """
    Functional wrapper for OrthogonalRegularizer.
    :param l1: l1 penalty
    :param l2: l2 penalty
    :return: Orthogonal regularizer to append to a loss function
    """
    return OrthogonalRegularizer(l1, l2)


def transform_net(inputs, scope=None, regularize=False):
    """
    Generates an orthogonal transformation tensor for the input preprocessing
    :param inputs: tensor with input image (either BxNxK or BxNx1xK)
    :param scope: name of the grouping scope
    :param regularize: enforce orthogonality constraint
    :return: BxKxK tensor of the transformation
    """
    with K.name_scope(scope):

        input_shape = inputs.get_shape().as_list()
        k = input_shape[-1]

        net = conv1d_bn(inputs, num_filters=64, kernel_size=1, padding='valid',
                        use_bias=True, scope='tconv1')
        net = conv1d_bn(net, num_filters=128, kernel_size=1, padding='valid',
                        use_bias=True, scope='tconv2')
        net = conv1d_bn(net, num_filters=1024, kernel_size=1, padding='valid',
                        use_bias=True, scope='tconv3')

        net = GlobalMaxPooling1D(data_format='channels_last')(net)

        net = dense_bn(net, units=512, scope='tfc1', activation='relu')
        net = dense_bn(net, units=256, scope='tfc2', activation='relu')

        transform = Dense(units=k * k,
                          kernel_initializer='zeros', bias_initializer=Constant(np.eye(k).flatten()),
                          activity_regularizer=orthogonal(l2=0.001) if regularize else None)(net)

        transform = Reshape((k, k))(transform)

    return transform

### PointNet Architecture

In [ ]:
def pointnet(input_shape, classes):
    """
    PointNet Model definition for classification.
    :param input_shape: The point cloud shape
    :param classes: Number of classes in output.
    :return: PointNet Model for classification with `classes` classes.
    """
    # Generate input tensor
    inputs = # ToDo define Input-Layer

    # Obtain spatial point transform from inputs and convert inputs
    ptransform = transform_net(inputs, scope='transform_net1', regularize=False)
    point_cloud_transformed = # ToDo Perform the matrix multiply between ptransform and inputs

    # First block of convolutions
    net = # ToDo, define first Conv-Layer
    net = # ToDo, define second Conv-Layer

    # Obtain feature transform and apply it to the network
    ftransform = transform_net(net, scope='transform_net2', regularize=True)
    net_transformed = # ToDo Perform the matrix multiply between ftransform and net

    # Second block of convolutions
    net = # ToDo, define third Conv-Layer
    net = # ToDo, define fourth Conv-Layer
    net = # ToDo, define fifth Conv-Layer

    # add Maxpooling 
    net = # Todo define MaxPool Layer

    # Top layers
    net = # ToDo, define first Dense-Layer
    net = # ToDo, define first Dropout-Layer
    net = # ToDo, define second Dense-Layer
    net = # ToDo, define second Dropout-Layer
    net = # ToDo, define third Dense-Layer

    model = Model(inputs, net, name='pointnet')

    return model

## Part Three: Transfer Learning of the PointNet

Load the pretrained weigths for PointNet and change the output to four classes. Define all Hyperparameters and implement the train/test and your own learning rate scheduling algorithm. 

In [ ]:
pretrained_model = pointnet((None, 3), 6)
pretrained_model.load_weights("pretrained-modelnet.h5")

# We need to replace the last layer, because it was trained to predict 6 classes instead of 4.
embedding_layer = # ToDo find "embedding layer" (The layer before the old softmax layer)
outputs = # ToDo Define new Output Layer
new_model = Model(inputs=pretrained_model.inputs, outputs=outputs)

In [ ]:
# TODO: Define Hyperparameters
SAMPLE_SIZE = 
EPOCHS = 
TRAIN_TEST_SPLIT = 
BATCH_SIZE =

In [ ]:
def random_split(samples, atFraction):
    """
    Perform the train/test split.
    """
    print("atFraction = ", atFraction)
    
    # ToDo perform the train/test split.
    pass


def reduce_learning_rate(epoch, currentLearningRate):
    """
    Implements adaptive learning rate scheduling.
    """
    # ToDo define your own learning rate schedule.
    return currentLearningRate

In [ ]:
df, l_idx = initialize_dataset("ModelNet10/")

train_samples, validation_samples = random_split(df[df["is_train"]], TRAIN_TEST_SPLIT)
test_samples = df[~df["is_train"]]

print("Number of training samples: ", len(train_samples))
print("Number of validation samples: ", len(validation_samples))
print("Number of hold out test samples: ", len(test_samples))

generator_training = ModelNetProvider(train_samples, batch_size=BATCH_SIZE, n_classes=4, sample_size=SAMPLE_SIZE)
generator_validation = ModelNetProvider(validation_samples, batch_size=BATCH_SIZE, n_classes=4, sample_size=SAMPLE_SIZE)


new_model.compile(optimizer="adam", loss='categorical_crossentropy',
              metrics=["categorical_accuracy"])

callbacks = [
    keras.callbacks.LearningRateScheduler(reduce_learning_rate, verbose=1),
    keras.callbacks.TensorBoard("logs/", batch_size=BATCH_SIZE),
    keras.callbacks.ModelCheckpoint("checkpoints/weights.{epoch:03d}-{val_loss:.2f}.h5", save_weights_only=True, save_best_only=True),
]

In [ ]:
%tensorboard --logir ....

In [ ]:
# train model
new_model.fit_generator(generator=generator_training, epochs=EPOCHS, callbacks=callbacks, 
                    validation_data=generator_validation, use_multiprocessing=False)

In [ ]:
# validate model
generator_test = ModelNetProvider(test_samples, batch_size=1, n_classes=4, sample_size=SAMPLE_SIZE)

val_df = pd.DataFrame({"prediction": [], "ground_truth": []})
for i in range(len(generator_test)):
    X, y = generator_test[i]
    prediction = new_model.predict(X)
    val_df = val_df.append({"prediction": l_idx[np.argmax(prediction)],  "ground_truth": l_idx[np.argmax(y)]}, 
                  ignore_index=True)

In [ ]:
from sklearn import metrics
import seaborn as sns
from ipywidgets import interact
import matplotlib.pyplot as plt
sns.set_context('talk')
palette = sns.color_palette(n_colors=6)

cm = metrics.confusion_matrix(val_df.ground_truth, val_df.prediction)
confusionMatrix = pd.DataFrame(cm, index=l_idx, columns=l_idx)
plt.figure(figsize=(10, 8))

sns.heatmap(confusionMatrix, vmin=0, annot=True, fmt="d", linewidth=.5, mask=(confusionMatrix==0), cmap="viridis")
plt.xlabel("Prediction")
plt.ylabel("Truth")
plt.ylim(4, 0)